#**Pre-request**

##Mount google drive


In [ ]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install pakages


In [ ]:
# Install packages
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
!pip install -q -r "{project_path}requirement/Install/Extract_requirements.txt" --no-cache-dir
%cd $project_path
%ls $project_path

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
configs/  notebooks/  requirement/  run_experiment.py  src/
dataset/  README.md   results/      splits/            tests/


##Import  libs

In [ ]:

import datetime
import os
import pandas as pd
import numpy as np
from scipy.stats import mode
import yaml
import logging
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import altair as alt
from google.colab import data_table
data_table.DataTable.MAX_COLUMNS = 100
data_table.DataTable.MAX_ROWS = 1000000
data_table.disable_dataframe_formatter()
data_table.enable_dataframe_formatter()
# Expand Colab’s table display limits
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
print("MAX_COLUMNS =", data_table.DataTable.MAX_COLUMNS)
print("MAX_COLUMNS =", data_table.DataTable.MAX_ROWS)
from datetime import datetime
from collections import deque



MAX_COLUMNS = 100
MAX_COLUMNS = 1000000


#Utility Functions

##Config

In [ ]:

def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file and expand ${root_path} placeholders."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)


    # --- Expand ${root_path} placeholders ---
    root = config.get("root_path", "")

    def expand_paths(obj):
        if isinstance(obj, dict):
            return {k: expand_paths(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [expand_paths(i) for i in obj]
        elif isinstance(obj, str) and "${root_path}" in obj:
            return obj.replace("${root_path}", root)
        else:
            return obj

    config = expand_paths(config)
    return config

## CDR dataset

In [ ]:
def load_cdr(file_path, nrows=None):
    """Load a CSV file and safely parse datetime columns."""
    df = pd.read_csv(file_path, nrows=nrows)

    # Auto-detect and parse datetime columns
    for col in df.columns:
        if "datetime" in col.lower() or "time" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

    df.columns = df.columns.str.strip()  # clean header spaces
    return df


def load_all_data(config):
    """
    Load all CSVs defined in config['Agg'] into a dict of DataFrames.
    """
    agg_cfg = config["Agg"]
    base = agg_cfg["input_path"]
    files = agg_cfg["files"]

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path)
        data[name] = df
    return data

##Features

###Voice Features

In [ ]:
def get_voc_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user voice call features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (moved from build_user_snapshots)
    if cutoff_time is not None:
        df = df[df["start_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("start_datetime").tail(n_events)

    df["call_dur"] = pd.to_numeric(df["call_dur"], errors="coerce").fillna(0)

    df["weekday"] = pd.to_datetime(df["start_datetime"]).dt.weekday
    df["hour"] = pd.to_datetime(df["start_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            voc_total_calls=("start_datetime", "count"),
            voc_unique_contacts=("opposite_no_m", "nunique"),
            voc_total_duration=("call_dur", "sum"),
            voc_avg_duration=("call_dur", "mean"),
            voc_max_duration=("call_dur", "max"),
            voc_std_duration=("call_dur", "std"),
            voc_active_days=("weekday", "nunique"),
            voc_active_hours=("hour", "nunique"),
        )
    )
    return feats.fillna(0)


###SMS Features

In [ ]:
def get_sms_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user SMS features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (if requested)
    if cutoff_time is not None:
        df = df[df["request_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("request_datetime").tail(n_events)

    # ✅ Ensure calltype_id is numeric
    df["calltype_id"] = pd.to_numeric(df["calltype_id"], errors="coerce")

    # ✅ Extract hour for time-based features
    df["hour"] = pd.to_datetime(df["request_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            sms_total_msgs=("request_datetime", "count"),
            sms_unique_contacts=("opposite_no_m", "nunique"),
            sms_active_hours=("hour", "nunique"),
            # 1 → outgoing, 2 → incoming (adjust if opposite)
            sms_calltype_ratio=("calltype_id", lambda x: (x == 1).mean()),
        )
    )

    return feats.fillna(0)


###App Features

In [ ]:
import pandas as pd
import datetime

def get_app_feats(df):
    """Extract per-user application usage features (aggregated monthly).
    Handles various month_id formats internally (201908, '2019-08', '2019/08', '2019-12-01').
    """
    df = df.copy()

    if df.empty:
        return pd.DataFrame(columns=[
            "phone_no_m", "app_months_active", "app_total_flow",
            "app_avg_flow", "app_std_flow",
            "app_unique_apps_mean", "app_unique_apps_max"
        ])

    # Ensure month_id exists
    if "month_id" not in df.columns:
        raise ValueError("❌ APP dataset must contain 'month_id' column.")

    # Convert flow to numeric
    df["flow"] = pd.to_numeric(df["flow"], errors="coerce").fillna(0)

    # Inline month_id parser
    def parse_month_end(x):
        if pd.isna(x):
            return pd.NaT
        s = str(x).strip()
        # Handle YYYYMM
        if s.isdigit() and len(s) == 6:
            y, m = int(s[:4]), int(s[4:])
            return pd.Timestamp(datetime.date(y, m, 1)) + pd.offsets.MonthEnd(0)
        # Handle full or partial date strings
        for fmt_try in [s, s + "-01"]:
            try:
                dt = pd.to_datetime(fmt_try, errors="coerce")
                if pd.notna(dt):
                    return dt + pd.offsets.MonthEnd(0)
            except Exception:
                continue
        return pd.NaT

    # Apply month parsing
    df["month_end"] = df["month_id"].apply(parse_month_end)
    df = df.dropna(subset=["month_end"])

    # Aggregate monthly totals
    monthly = (
        df.groupby(["phone_no_m", "month_end"])
        .agg(
            total_flow=("flow", "sum"),
            unique_apps=("busi_name", "nunique"),
        )
        .reset_index()
    )

    # Aggregate per-user statistics
    features = (
        monthly.groupby("phone_no_m")
        .agg(
            app_months_active=("month_end", "nunique"),
            app_total_flow=("total_flow", "sum"),
            app_avg_flow=("total_flow", "mean"),
            app_std_flow=("total_flow", "std"),
            app_unique_apps_mean=("unique_apps", "mean"),
            app_unique_apps_max=("unique_apps", "max"),
        )
        .reset_index()
        .fillna(0)
    )

    return features


###User Features

In [ ]:
def get_user_feats(df):
    """
    Extract per-user ARPU-based features.
    Handles both YYYYMM and YYYY-MM-DD month_id formats.
    Zeros are considered inactive unless explicitly kept.
    """

    import pandas as pd
    df = df.copy()

    if "arpu_value" not in df.columns:
        raise ValueError("Expected column 'arpu_value' not found.")

    # --- Convert month_id to datetime safely ---
    def to_month_end(val):
        if pd.isna(val):
            return pd.NaT
        s = str(val).strip()
        # Handle both '2019-08-01' and '201908'
        try:
            dt = pd.to_datetime(s, errors="coerce")
            if pd.notna(dt):
                return dt + pd.offsets.MonthEnd(0)
        except Exception:
            pass
        # fallback for YYYYMM numeric
        s = s.replace("-", "").replace("/", "")
        if len(s) == 6:
            try:
                return pd.to_datetime(s + "01", format="%Y%m%d") + pd.offsets.MonthEnd(0)
            except Exception:
                return pd.NaT
        return pd.NaT

    df["month_end"] = df["month_id"].apply(to_month_end)

    # --- Convert ARPU values to numeric ---
    df["arpu_value"] = pd.to_numeric(df["arpu_value"], errors="coerce")

    # --- Filter valid ARPU entries ---
    df_valid = df[df["arpu_value"].notna() & (df["arpu_value"] > 0)]

    # --- Aggregate per user ---
    user_feats = (
        df_valid.groupby("phone_no_m", as_index=False)
        .agg(
            user_months_active=("month_end", "nunique"),
            arpu_mean=("arpu_value", "mean"),
            arpu_std=("arpu_value", "std"),
            arpu_max=("arpu_value", "max"),
            idcard_cnt=("idcard_cnt", "max"),
            label=("label", "max"),
        )
    )

    # --- If no active month found, return zero row ---
    if user_feats.empty:
        user_feats = pd.DataFrame([{
            "phone_no_m": df["phone_no_m"].iloc[0],
            "user_months_active": 0,
            "arpu_mean": 0,
            "arpu_std": 0,
            "arpu_max": 0,
            "idcard_cnt": df.get("idcard_cnt", [0])[0],
            "label": df.get("label", [0])[0],
        }])

    return user_feats


###Get feature names

In [ ]:
def get_feature_names():
    ALL_FEATURE_COLUMNS = [
        # Voice
        "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
        "voc_avg_duration", "voc_max_duration", "voc_std_duration",
        "voc_active_days", "voc_active_hours",
        # SMS
        "sms_total_msgs", "sms_unique_contacts", "sms_active_hours", "sms_calltype_ratio",
        # App
        "app_months_active", "app_total_flow", "app_avg_flow",
        "app_std_flow", "app_unique_apps_mean", "app_unique_apps_max",
        # User / ARPU
        "user_months_active", "arpu_mean", "arpu_std", "arpu_max",
        "idcard_cnt", "label"
    ]
    return ALL_FEATURE_COLUMNS


##Snapshot

###ensure dataframe

In [ ]:

def ensure_dataframe(df, label, user):
    """Guarantee a DataFrame with phone_no_m even if empty or Series."""
    if df is None:
        df = pd.DataFrame()
    if isinstance(df, pd.Series):
        df = df.to_frame().T
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)
    if df.empty or "phone_no_m" not in df.columns:
        df["phone_no_m"] = [user]
    return df.reset_index(drop=True)



###Feature extraction wrapper

In [ ]:
def extract_features_for_sources(subsets, user):
    """Run feature extraction for each data source safely."""
    feats = {}
    try:
        feats["USER"] = get_user_feats(subsets["ARPU"]) if not subsets["ARPU"].empty else pd.DataFrame()
        feats["VOC"]  = get_voc_feats(subsets["VOC"])   if not subsets["VOC"].empty else pd.DataFrame()
        feats["SMS"]  = get_sms_feats(subsets["SMS"])   if not subsets["SMS"].empty else pd.DataFrame()
        feats["APP"]  = get_app_feats(subsets["APP"])   if not subsets["APP"].empty else pd.DataFrame()
    except Exception as e:
        print(f"❌ Feature extraction failed for user {user}: {e}")
        return {}

    # Ensure dataframes are valid and have phone_no_m
    for key in feats:
        feats[key] = ensure_dataframe(feats[key], key.lower(), user)
    return feats


###Combine to single snapshot row

In [ ]:
def combine_features_to_snapshot(
    user, step, cutoff_time, event_type,
    window_mode, window_size, feats,
    all_feature_columns, start_window
):
    """Merge all per-source features into one unified snapshot row."""
    base = pd.DataFrame({
        "phone_no_m": [user],
        "snapshot_index": [step],
        "snapshot_time": [cutoff_time],
        "event_type": [event_type],
        "window_mode": [window_mode],
        "window_size": [window_size],
        "window_start": [start_window],  # ✅ added here
        "window_end": [cutoff_time]      # ✅ optional clarity
    })

    snapshot = (
        base
        .merge(feats.get("VOC", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("SMS", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("APP", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("USER", pd.DataFrame()), on="phone_no_m", how="left")
    )

    snapshot = snapshot.reindex(
        columns=[
            "phone_no_m", "snapshot_index", "snapshot_time",
            "window_start", "window_end", "event_type",
            "window_mode", "window_size"
        ] + all_feature_columns,
        fill_value=0
    )
    return snapshot


###Time-Aware User Snapshots

In [ ]:

def build_user_snapshots_global(
    df_voc, df_sms, df_app_tx, df_arpu_tx,
    window_size=1, window_unit="days",
    window_mode="time", max_users=None, max_snapshots=None,
    debug=True
):
    from datetime import datetime

    snapshot_count=0
    """Round-by-round gradual snapshotting: add one event per user per round,
    printing selected event, current queue, and generated snapshot each round."""

    # 🧩 Merge & sort
    print(f"{datetime.now():%Y-%m-%d %H:%M:%S}- Merging started")
    all_events = pd.concat([df_voc, df_sms, df_app_tx, df_arpu_tx], ignore_index=True)
    print(f"{datetime.now():%Y-%m-%d %H:%M:%S} - Total events: {len(all_events)} Merging Ended")
    print(f"{datetime.now():%Y-%m-%d %H:%M:%S} - Total events: {len(all_events)} Sorting started")
    all_events = all_events.sort_values(["phone_no_m", "event_time"]).reset_index(drop=True)
    print(f"{datetime.now():%Y-%m-%d %H:%M:%S} - Total events: {len(all_events)} Sorting end")
    users = df_arpu_tx["phone_no_m"].unique()
    if max_users:
        users = users[:max_users]

    print(f"Total users: {len(users)}")

    # 🧩 Initialize per-user state


    from collections import deque

    grouped = all_events.groupby("phone_no_m", sort=False)

    user_data = {
        u: {
            "events": g.reset_index(drop=True),
            "queue": deque(),
            "done": False
        }
        for u, g in grouped
        if u in users  # optional safety check
    }

    ##display(user_data)
    all_snapshots = []
    round_index = 0


# 📁 Prepare output path and timestamped filename (once for the whole run)
    save_path = config["Agg"]["save_path"]
    os.makedirs(save_path, exist_ok=True)
    run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = os.path.join(save_path, f"user_snapshots_{run_timestamp}.csv")

    print(f"📦 Snapshot run started — output file: {output_file}")


    # 🌀 Begin round-robin gradual processing
    while True:
        round_snapshots = []
        selected_events = []
        active = False

        #print(f"\n==============================")
       # print(f"🚀 ROUND {round_index}")
        #print(f"==============================")

        for u in users:
            ud = user_data[u]
            df = ud["events"]
            q = ud["queue"]

            if ud["done"] or df.empty:
                continue
            active = True

            # ✅ Take one event for this user
            event = df.iloc[0]
            ud["events"] = df.iloc[1:].reset_index(drop=True)
            q.append(event)
            current_time = event["event_time"]

            # Trim queue according to window
            if window_mode == "events":
                while len(q) > window_size:
                    q.popleft()
            elif window_mode == "time":
                while (
                    q and (current_time - q[0]["event_time"]) > pd.Timedelta(**{window_unit: window_size})
                ):
                    q.popleft()

            if not q:
                ud["done"] = True
                continue

            # 🧾 Selected event
            selected_events.append({
                "phone_no_m": u,
                "event_time": event["event_time"],
                "source": event["source"],
                "queue_size": len(q)
            })

            # 🪣 Print selected event and queue
            if debug:
                print(f"\n📥 User: {u}")
                print(f"   ➕ Added event: {event['event_time']} ({event['source']})")
                print(f"   🧮 Current queue ({len(q)} events):")
                qdf_display = pd.DataFrame(list(q))[["source", "event_time"]]
                #display(qdf_display)

            # Build snapshot
            qdf = pd.DataFrame(list(q))
            window_start = qdf["event_time"].min()
            window_end = qdf["event_time"].max()
            event_type = event["source"]

            subsets = {
                "VOC": qdf[qdf["source"] == "VOC"],
                "SMS": qdf[qdf["source"] == "SMS"],
                "APP": qdf[qdf["source"] == "APP"],
                "ARPU": qdf[qdf["source"] == "ARPU"],  # match extractor naming
            }

            feats = extract_features_for_sources(subsets, u)
            snapshot = combine_features_to_snapshot(
                user=u,
                step=round_index,
                cutoff_time=window_end,
                event_type=event_type,
                window_mode=window_mode,
                window_size=window_size,
                feats=feats,
                all_feature_columns=get_feature_names(),
                start_window=window_start
            )

            snapshot["window_start"] = window_start
            snapshot["window_end"] = window_end
            snapshot["snapshot_round"] = round_index
            round_snapshots.append(snapshot)

            if ud["events"].empty:
                ud["done"] = True

        # 🛑 Stop if no users active
        if not active:
            print("\n✅ All users processed — exiting.")
            break

        # 🧾 Show selected events summary
        if selected_events:
            #print(f"\n🔹 Selected Events Summary for Round {round_index}")
            sel_df = pd.DataFrame(selected_events)
            #display(sel_df[["phone_no_m", "event_time", "source", "queue_size"]])
        else:
            print("(No events selected this round)")

        # 📸 Show snapshot for this round
        if round_snapshots:
          round_df = pd.concat(round_snapshots, ignore_index=True)
          all_snapshots.append(round_df)

          # ✅ Attach labels for this round
          global_labels = (
              df_user[['phone_no_m', 'label']]
              .dropna(subset=['phone_no_m'])
              .drop_duplicates('phone_no_m')
          )
          round_df = (
              round_df
              .drop(columns=['label'], errors='ignore')
              .merge(global_labels, on='phone_no_m', how='left')
          )

          # ✅ Append this round to the same file
          header_needed = not os.path.exists(output_file)
          #round_df.to_csv(output_file, mode='a', index=False, header=header_needed)
          # 💡 Add these at the top of your loop (before it starts)
          batch_buffer = []
          batch_size = 50  # write every 5 rounds

          # ...
          # inside your snapshot loop:
          batch_buffer.append(round_df)

          # every N rounds → write accumulated data
          if (round_index + 1) % batch_size == 0 or (round_index + 1) == max_snapshots:
              batch_df = pd.concat(batch_buffer, ignore_index=True)
              batch_df.to_csv(output_file, mode='a', index=False, header=header_needed)
              batch_buffer.clear()  # free memory
              header_needed = False  # only needed for the first write
              print(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] 💾 Appended snapshot round {round_index} → {output_file}")
              #print(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] 💾 Appended snapshot round {round_index} → {output_file}")


        round_index += 1

        if round_index % 5 == 0:
            print(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] | Current snapshot is {round_index}")


        if max_snapshots and round_index >= max_snapshots:
            print("⛔ Reached max snapshot limit.")
            break

    print("\n✅ Completed all snapshot rounds.")
    return pd.concat(all_snapshots, ignore_index=True) if all_snapshots else pd.DataFrame()


#Excute pipeline

##Config and summary

In [ ]:
# 1️⃣ Load config and all datasets
config = load_config(os.path.join(project_path, "configs", "baseline.yaml"))
data = load_all_data(config)


# 2️⃣ Extract individual datasets from the returned dictionary
df_voc = data["voc"]
df_sms = data["sms"]
df_app = data["app"]
df_user = data["user"]


# Voice
df_voc["source"] = "VOC"
df_voc["event_time"] = pd.to_datetime(df_voc.get("start_datetime", df_voc.get("event_time")), errors="coerce")

# SMS
df_sms["source"] = "SMS"
df_sms["event_time"] = pd.to_datetime(df_sms.get("request_datetime", df_sms.get("event_time")), errors="coerce")

# App
df_app["source"] = "APP"
df_app["event_time"] = pd.to_datetime(df_app["event_time"], errors="coerce")

# ARPU (User)
df_user["source"] = "ARPU"
df_user["event_time"] = pd.to_datetime(df_user["event_time"], errors="coerce")

print("✅ All datasets standardized and ready for timeline merge:")
print(f"  VOC  → {len(df_voc):,} records")
print(f"  SMS  → {len(df_sms):,} records")
print(f"  APP  → {len(df_app):,} records")
print(f"  ARPU → {len(df_user):,} records")

for df, label in [(df_user, "VOC"), (df_sms, "SMS"), (df_app, "APP"), (df_user, "ARPU")]:
            # Normalize event_time field
            if "event_time" not in df.columns or df["event_time"].isna().all():
                for alt_col in ["start_datetime", "request_datetime", "date", "busi_date"]:
                    if alt_col in df.columns:
                        df["event_time"] = pd.to_datetime(df[alt_col], errors="coerce")
                        break


✅ All datasets standardized and ready for timeline merge:
  VOC  → 5,015,430 records
  SMS  → 6,848,509 records
  APP  → 3,283,602 records
  ARPU → 39,454 records


##Explore

In [ ]:
import numpy as np
import pandas as pd

# =========================
# 1️⃣ Extract fraud labels
# =========================
# Assuming df_user contains a column 'label' (1 = fraud, 0 = non-fraud)
user_labels = df_user.groupby("phone_no_m")["label"].first()  # ensure unique per user

# 🧠 Function to compute per-user event counts for any dataframe
def get_event_counts(df, user_col="phone_no_m", time_col="event_time"):
    if df is None or df.empty:
        return pd.Series(dtype=int)
    return df.groupby(user_col)[time_col].size()

# =========================
# 2️⃣ Compute counts for each source
# =========================
v_counts = get_event_counts(df_voc)
s_counts = get_event_counts(df_sms)
a_counts = get_event_counts(df_app)
u_counts = get_event_counts(df_user)

# 🧩 Align all users (union across all data sources)
all_users = (
    v_counts.index
    .union(s_counts.index)
    .union(a_counts.index)
    .union(u_counts.index)
    .union(user_labels.index)
)

v_counts = v_counts.reindex(all_users, fill_value=0)
s_counts = s_counts.reindex(all_users, fill_value=0)
a_counts = a_counts.reindex(all_users, fill_value=0)
u_counts = u_counts.reindex(all_users, fill_value=0)
user_labels = user_labels.reindex(all_users, fill_value=np.nan)

# =========================
# 3️⃣ Combine counts (union of all events)
# =========================
events_per_user = v_counts + s_counts + a_counts + u_counts

# Attach fraud label
events_df = pd.DataFrame({
    "total_events": events_per_user,
    "fraud_label": user_labels
})

# Drop users with no label (if any)
events_df = events_df.dropna(subset=["fraud_label"])
events_df["fraud_label"] = events_df["fraud_label"].astype(int)

# =========================
# 4️⃣ Compute per-class statistics
# =========================
for label_value, group in events_df.groupby("fraud_label"):
    label_name = "Fraud" if label_value == 1 else "Non-Fraud"
    print(f"\n📊 === {label_name} Users ===")
    print(f"Users: {len(group):,}")
    print(group["total_events"].describe(percentiles=[0.5, 0.9, 0.91, 0.95, 0.99]).to_string())

    cap_91 = int(np.percentile(group["total_events"], 91))
    print(f"🚀 Suggested cap_91 for {label_name}: {cap_91} snapshots/user (approx.)")

# =========================
# 5️⃣ Global (all users)
# =========================
print("\n🌍 === Global (All Users) ===")
print(events_df["total_events"].describe(percentiles=[0.5, 0.9, 0.91, 0.95, 0.99]).to_string())



📊 === Non-Fraud Users ===
Users: 4,141
count     4141.000000
mean      2763.943250
std       2444.189427
min         21.000000
50%       2142.000000
90%       5760.000000
91%       5929.000000
95%       7176.000000
99%      11578.000000
max      31032.000000
🚀 Suggested cap_91 for Non-Fraud: 5929 snapshots/user (approx.)

📊 === Fraud Users ===
Users: 1,788
count     1788.00000
mean      2029.80481
std       3141.35987
min          3.00000
50%        556.00000
90%       6321.40000
91%       6675.57000
95%       8794.90000
99%      14106.43000
max      28258.00000
🚀 Suggested cap_91 for Fraud: 6675 snapshots/user (approx.)

🌍 === Global (All Users) ===
count     5929.000000
mean      2542.550177
std       2694.524982
min          3.000000
50%       1735.000000
90%       5879.200000
91%       6076.400000
95%       7609.600000
99%      12394.440000
max      31032.000000


In [ ]:
# choose anchor df: df_voc / df_sms / df_app (or merged)
events_per_user = df_voc.groupby("phone_no_m")["event_time"].size()

cap_95 = int(np.percentile(events_per_user, 95))   # e.g., 95th percentile
max_snapshots = min(cap_95, 200)  #Not used                 # safety upper bound
print(f"Max snapshots per user: {max_snapshots}")



Max snapshots per user: 200


##Genrate snapshot

In [ ]:

max_users=5929
max_snapshots=100 #overide
window_size=10
window_unit="days"
window_mode="time"
snapshots_df = build_user_snapshots_global(
    df_voc=df_voc,
    df_sms=df_sms,
    df_app_tx=df_app,
    df_arpu_tx=df_user,
    window_size=window_size,
    window_unit=window_unit,
    window_mode=window_mode,
   # max_users=max_users,
    max_snapshots=max_snapshots,
    debug=False
)

# ✅ Attach global labels (fixes snapshot label mismatch)
global_labels = (
    df_user[['phone_no_m', 'label']]
    .dropna(subset=['phone_no_m'])
    .drop_duplicates('phone_no_m')
)
snapshots_df = (
    snapshots_df
    .drop(columns=['label'], errors='ignore')
    .merge(global_labels, on='phone_no_m', how='left')
)
#display(snapshots_df)
# Save output snapshot
save_path = config["Agg"]["save_path"]
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(save_path, exist_ok=True)

output_file = os.path.join(save_path, f"user_snapshots_{max_snapshots}_users_{max_users}_mode_{window_mode}_size_{window_size}_{timestamp}.csv")
snapshots_df.to_csv(output_file, index=False)

print(f"✅ Snapshot file saved to: {output_file}")

2025-10-25 14:04:18- Merging started
2025-10-25 14:04:23 - Total events: 15186995 Merging Ended
2025-10-25 14:04:23 - Total events: 15186995 Sorting started
2025-10-25 14:04:43 - Total events: 15186995 Sorting end
Total users: 5929
📦 Snapshot run started — output file: /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/snapshots/full/user_snapshots_20251025_140451.csv
[2025-10-25 14:14:08] | Current snapshot is 5
[2025-10-25 14:24:19] | Current snapshot is 10
[2025-10-25 14:34:53] | Current snapshot is 15
[2025-10-25 14:45:54] | Current snapshot is 20
[2025-10-25 14:57:16] | Current snapshot is 25
[2025-10-25 15:08:53] | Current snapshot is 30
[2025-10-25 15:21:05] | Current snapshot is 35
[2025-10-25 15:33:19] | Current snapshot is 40
[2025-10-25 15:45:48] | Current snapshot is 45
[2025-10-25 15:58:30] 💾 Appended snapshot round 49 → /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/dataset/CallChinses/snapshots/full/user_snapshots_20251025_140451.c

#freeze

In [ ]:
%pip freeze > "{project_path}requirement/freez/SequenceStreaming_requirements-lock.txt"
